# Association Analysis of Sequence Data using Variant Association Tools (VAT) for Complex Traits

Copyright (c) 2018, Gao Wang, Biao Li, Diana Cornejo Sánchez & Suzanne M. Leal

## PURPOSE
Variant Association Tools [VAT, Wang et al (2014)] [1] was developed to perform quality control and association analysis of sequence data. It can also be used to analyze genotype data, e.g. exome chip data and imputed data. The software incorporates many rare variant association methods which include but not limited to Combined Multivariate Collapsing (CMC) [2], Burden of Rare Variants (BRV) [3], Weighted Sum Statistic (WSS) [4], Kernel Based Adaptive Cluster (KBAC) [5], Variable Threshold (VT) [6] and Sequence Kernel Association Test (SKAT) [7].
VAT inherits the intuitive command-line interface of Variant Tools (VTools) [8] with re-design and implementation of its infrastructure to accommodate the scale of dataset generated from current sequencing efforts on large populations. Features of VAT are implemented into VTools subcommand system.

## RESOURCES

A list of all commands that are used in this exercise can be found at:

https://statgen.research.bcm.edu/index.php/Main_Page

Basic concepts to handle sequence data using vtools can be found at:

http://varianttools.sourceforge.net/Main/Concepts

VAT Software documentation:

http://varianttools.sourceforge.net/Main/Documentation

### Genotype data

Exome genotype data was downloaded from the 1000 Genomes pilot data July 2010 release for both the CEU and YRI populations. Only the autosomes are contained in the datasets accompanying this exercise.
The data sets (`CEU.exon.2010_03.genotypes.vcf.gz, YRI.exon.2010_03.genotypes.vcf.gz`) are available from:
ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/pilot_data/release/2010_07/exon/snps

### Phenotype data
To demonstrate the association analysis, we simulated a quantitative trait phenotype (BMI). Please note that these phenotypes are NOT from the 1000 genome project.

### Computation resources
Due to the nature of next-generation sequencing data, a reasonably powerful machine with high speed internet connection is needed to use this tool for real-world applications. For this reason, in this tutorial we will use a small demo dataset to demonstrate association analysis.

## Part I: Data Quality Control, Annotation and Variant/sample Selection

### 1.1	Getting started
Please navigate to the exercise data directory,

In [1]:
%cd data

/home/gaow/tmp/31-May-2018/data

and check the available subcommands by typing:

In [2]:
vtools -h

usage: vtools [-h] [--version]
              {init,import,phenotype,show,liftover,use,update,select,exclude,compare,output,export,remove,associate,admin,execute}
              ...

A variant calling, processing, annotation and analysis tool for next-
generation sequencing studies.

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit

subcommands:
  {init,import,phenotype,show,liftover,use,update,select,exclude,compare,output,export,remove,associate,admin,execute}
    init                Create a new project, or a subproject from an existing
                        parent project, or merge several existing projects
                        into one
    import              Import variants and related sample genotype from files
                        in specified formats
    phenotype           Manage sample phenotypes
    show                Display content of a project
    liftover            Set alte

Subcommand system is used for various data manipulation tasks (to check details of each subcommand use `vtools <name of subcommand> -h`). This tutorial is mission oriented and focuses on a subset of the commands that are relevant to variant-phenotype association analysis, rather than introducing them systematically. For additional functionality, please refer to documentation and tutorials online.

### Initialize a project

In [6]:
vtools init VATDemo

INFO: variant tools 3.0.0dev : Copyright (c) 2011 - 2016 Bo Peng
INFO: Please visit http://varianttools.sourceforge.net for more information.
INFO: Creating a new project VATDemo


Command `vtools init` creates a new project in the current directory. A directory can only have one project. After a project is created, subsequent `vtools` calls will automatically load the project in the current directory. Working from outside of a project directory is not allowed.

### Import variant and genotype data


Import all vcf files under the current directory:


In [7]:
vtools import *.vcf.gz --var_info DP filter --geno_info DP_geno --build hg18 -j1

INFO: Importing variants from CEU.exon.2010_03.genotypes.vcf.gz (1/2)
CEU.exon.2010_03.genotypes.vcf.gz: 100% [============] 4,306 19.0K/s in 00:00:00
INFO: 3,489 new variants (3,489 SNVs) from 3,500 lines are imported.
Importing genotypes: 100% [===========================] 3,489 7.0K/s in 00:00:00
INFO: Importing variants from YRI.exon.2010_03.genotypes.vcf.gz (2/2)
YRI.exon.2010_03.genotypes.vcf.gz: 100% [============] 5,967 19.5K/s in 00:00:00
INFO: 3,498 new variants (5,175 SNVs) from 5,186 lines are imported.
Importing genotypes: 100% [==========================] 6,987 10.4K/s in 00:00:00


Command `vtools import` imports variants, sample genotypes and related information fields. The imported variants are saved to the master variant table for the project, along with their information fields.

The command above imports two vcf files sequentially into an empty `vtools` project. The second `INFO` message in the screen output shows that 3,489 variant sites are imported from the first vcf file, where 3,489 new means that all of them are new because prior to importing the first vcf the project was empty so there was 0 site. The fourth `INFO` message tells that 5,175 variant sites are imported from the second vcf file, but only 3,498 of them are new (which are not seen in the existing 3,489) because prior to importing the second vcf there were already 3,489 existing variant sites from first vcf. 

Thus, 5,175 - 3,498 = 1,677 variant sites are overlapped sites between first and second vcfs. The last `INFO` message summarizes that the sum of variant sites contained in both vcfs is 8,664 = 3,489 + 5,175, where there are 6,987 variant sites after merging variants from both vcfs.

More details about `vtools import` command can be found at:

http://varianttools.sourceforge.net/Vtools/Import

Since the input VCF file uses hg18 as the reference genome while most modern annotation data sources are hg19-based, we need to "liftover" our project using hg19 in order to use various annotation sources in the analysis. Vtools provides a command which is based on the tool of USCS `liftOver` to map the variants from existing reference genome to an alternative build. More details about `vtools liftover` command can be found at:

http://varianttools.sourceforge.net/Vtools/Liftover

In [10]:
vtools liftover hg19

INFO: Downloading liftOver chain file from UCSC
INFO: Exporting variants in BED format
Exporting variants: 100% [==========================] 6,987 200.9K/s in 00:00:00
INFO: Running UCSC liftOver tool
Updating table variant: 100% [=======================] 6,987 989.6/s in 00:00:07


### Import phenotype data

The aim of the association test is to find variants that modulate the phenotype BMI. We simulated BMI values for each of the individuals. The phenotype file must be in plain text format with sample names matching the sample IDs in the vcf file(s):

In [11]:
head phenotypes.csv

head: cannot open 'phenotypes.csv' for reading: No such file or directory


: 1

The phenotype file includes information for every individual, the sample name, sequencing panel, sex and BMI. To import the phenotype data:

In [ ]:
vtools phenotype --from_file phenotypes.csv --delimiter ","

Unlike `vtools import`, this command imports/adds properties to samples rather than to variants. More details about `vtools phenotype` command can be found at:

http://varianttools.sourceforge.net/Vtools/Phenotype

### View imported data
Summary information for the project can be viewed anytime using the command `vtools show`, which displays various project and system information. More details about `vtools show` can be found at:

http://varianttools.sourceforge.net/Vtools/Show

Some useful data summary commands are:

In [12]:
vtools show project

Project name:                VATDemo
Created on:                  Fri Jun 29 07:55:30 2018
Primary reference genome:    hg18
Secondary reference genome:  hg19
Storage method:              hdf5
Runtime options:             verbosity=1, shared_resource=/home/gaow/.variant_tools, local_resource=/home/gaow/.variant_tools
Variant tables:              variant
Annotation databases:        



In [13]:
vtools show tables

table      #variants     date message
variant        6,987    Jun29 Master variant table


In [14]:
vtools show table variant

Name:                   variant
Description:            Master variant table
Creation date:          Jun29
Command:
Fields:                 variant_id, bin, chr, pos, ref, alt, DP, filter,
                        alt_bin, alt_chr, alt_pos
Number of variants:     6987


In [21]:
vtools show fields

variant.chr (char)      Chromosome name (VARCHAR)
variant.pos (int)       Position (INT, 1-based)
variant.ref (char)      Reference allele (VARCHAR, - for missing allele of an
                        insertion)
variant.alt (char)      Alternative allele (VARCHAR, - for missing allele of an
                        deletion)
variant.DP (int)
variant.filter (char)
variant.alt_chr (char)
variant.alt_pos (int)


## 1.2	Overview of variant and genotype data

### Total number of variants

The number of imported variants may be greater than number of lines in the vcf file, because when a variant has two alternative alleles (e.g. A->T/C) it is treated as two separate variants.

In [22]:
vtools select variant --count

Counting variants: 4 405.3/s in 00:00:00
6987


There are 6987 variants in our toy data-set.

`vtools select table condition action` selects from a variant table `table` a subset of variants satisfying a specified `condition`, and perform an `action` of

- creating a new variant table if `--to_table` is specified.
- counting the number of variants if `--count` is specified.
- outputting selected variants if `--output` is specified.

The `condition` should be a SQL expression using one or more fields in a project (displayed in `vtools show` fields). If the condition argument is unspecified, then all variants in the table will be selected. An optional condition `--samples [condition]` can also be used to limit selected variants to specific samples. More details about `vtools select` command can be found at:

http://varianttools.sourceforge.net/Vtools/Select

### Genotype Summary

The command `vtools show genotypes` displays the number of genotypes for each sample and names of the available genotype information fields for each sample, e.g. GT - genotype; DP geno - genotype read depth. Such information is useful for the calculation of summary statistics of genotypes (e.g. depth of coverage).

In [23]:
vtools show genotypes > genotype_summary.txt

In [25]:
%preview genotype_summary.txt

> genotype_summary.txt (11.5 KiB):

sample_name	filename                 	num_genotypes	sample_genotype_fields
NA06984    	CEU.exon...notypes.vcf.gz	3162         	DP,GT
NA06985    	CEU.exon...notypes.vcf.gz	3144         	DP,GT
NA06986    	CEU.exon...notypes.vcf.gz	3437         	DP,GT
NA06989    	CEU.exon...notypes.vcf.gz	3130         	DP,GT


## Variant Quality Overview

The following command calculates summary statistics on the variant site depth of coverage (DP). Below is the command to calculate depth of coverage information for all variant sites.